## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/Tim/Documents/OSU_Bootcamp/Challenge_6/Weather_Database/weatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,2022-01-04 03:22:34,-33.9258,18.4232,62.28,82,0,3.00,clear sky
1,1,Sao Filipe,CV,2022-01-04 03:23:39,14.8961,-24.4956,73.04,63,51,7.14,broken clouds
2,2,Santa Fe,AR,2022-01-04 03:23:39,-31.6333,-60.7000,82.02,64,7,1.99,light rain
3,3,Port Elizabeth,ZA,2022-01-04 03:23:39,-33.9180,25.5701,69.10,94,40,10.36,mist
4,4,Rikitea,PF,2022-01-04 03:23:40,-23.1203,-134.9692,78.08,67,100,4.38,overcast clouds


In [3]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
#temps


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [17]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

ValueError: Can only compare identically-labeled Series objects

In [ ]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")
    

        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [18]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.replace(r'^\s*$', np.nan, regex=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Sao Filipe,CV,73.04,broken clouds,14.8961,-24.4956,Hotel Xaguate
2,Santa Fe,AR,82.02,light rain,-31.6333,-60.7000,Conquistador Hotel
4,Rikitea,PF,78.08,overcast clouds,-23.1203,-134.9692,People ThankYou
5,Puerto Ayora,EC,71.24,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
12,Nha Trang,VN,80.60,overcast clouds,12.2500,109.1833,Sheraton Nha Trang Hotel & Spa
...,...,...,...,...,...,...,...
656,Iracoubo,GF,76.64,clear sky,5.4802,-53.2011,Le Gîte d’Iracoubo
658,Fare,PF,79.81,light rain,-16.7000,-151.0167,Maitai Lapita Village Huahine
660,Sechura,PE,70.59,overcast clouds,-5.5569,-80.8222,Hospedaje Costa Bella
661,Labuhan,ID,84.45,overcast clouds,-6.8844,112.2051,PT PPI Lamongan Tambak C


In [19]:
# 8a. Create the output File (CSV)
output_data_file = "/Users/Tim/Documents/OSU_Bootcamp/Challenge_6/Vacation_Search/weatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [20]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Weather</dt><dd>{Current Description}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
    """ 

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_data = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
print(len(hotel_data))
# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
print(len(locations))

222
222


In [21]:
#    info_box_template = """
#    <dl>
#    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
#    <dt>City</dt><dd>{City}</dd>
#    <dt>Country</dt><dd>{Country}</dd>
#    <dt>Weather</dt><dd>{Current Description}</dd>
#    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
#    </dl>
 #   """

In [22]:
hotel_data

['\n    <dl>\n    <dt>Hotel Name</dt><dd>Hotel Xaguate</dd>\n    <dt>City</dt><dd>Sao Filipe</dd>\n    <dt>Country</dt><dd>CV</dd>\n    <dt>Weather</dt><dd>broken clouds</dd>\n    <dt>Max Temp</dt><dd>73.04 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>Conquistador Hotel</dd>\n    <dt>City</dt><dd>Santa Fe</dd>\n    <dt>Country</dt><dd>AR</dd>\n    <dt>Weather</dt><dd>light rain</dd>\n    <dt>Max Temp</dt><dd>82.02 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>People ThankYou</dd>\n    <dt>City</dt><dd>Rikitea</dd>\n    <dt>Country</dt><dd>PF</dd>\n    <dt>Weather</dt><dd>overcast clouds</dd>\n    <dt>Max Temp</dt><dd>78.08 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n    <dt>City</dt><dd>Puerto Ayora</dd>\n    <dt>Country</dt><dd>EC</dd>\n    <dt>Weather</dt><dd>overcast clouds</dd>\n    <dt>Max Temp</dt><dd>71.24 °F</dd>\n    </dl>\n    ',
 '\n    <dl>\n    <dt>Hotel Name</dt><dd>Sherat

In [23]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp=clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)
marker_layer=gmaps.marker_layer(locations)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp=clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False, max_intensity=300, point_radius=4)

print(len(hotel_data))
print(len(locations))

# Corect the length difference between 

marker_layer=gmaps.marker_layer(locations,info_box_content=hotel_data)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)



# 11b. Display the figure
fig

222
222


Figure(layout=FigureLayout(height='420px'))